# Agents

An Agent is combination of an LLM model and some tools. The LLM act as brain of the Agent and Tools work as it's arms. 

In [1]:
## Setup LLM which we will use
from langchain_ollama.chat_models import ChatOllama
llm = ChatOllama(
    base_url = 'http://localhost:11434',
    model = 'qwen2.5:0.5b'
)

### Tools
Tools are like Python functions which can perform some action. 
  
**Note: LLM models cannot invoke a tool and give result. It is your responsibility to call tools. LLM can onlt tell which tools to be called**

In [4]:
### Lets try to create a simple Tool
from langchain.tools import tool

@tool
def my_first_tool(my_tool_arg: str) -> str:
    """This is description of Tool. And it is mandatory so that LLM can know what is purpose of the tool"""
    return "Some Output"

# Lets see information about these tools
print(f"{my_first_tool.name=}")
print(f"{my_first_tool.description=}")
print(f"{my_first_tool.args=}")

my_first_tool.name='my_first_tool'
my_first_tool.description='This is description of Tool. And it is mandatory so that LLM can know what is purpose of the tool'
my_first_tool.args={'my_tool_arg': {'title': 'My Tool Arg', 'type': 'string'}}


#### Lets try to create a sample tool

In [6]:
@tool
def big_crypto_coin(usd: float) -> float:
    "This tool converts the USD value to value for cyrpto currency names BIG and returns the amount in BIG currency"
    return 123.56*float(usd)

#### Lets bind the Tool with Our LLM

In [7]:
llm_with_crypto_tool = llm.bind_tools([big_crypto_coin])

In [10]:
# Lets ask the question and see what we get
result = llm_with_crypto_tool.invoke("How much BIG crypto can I get in 12 USD?")
print(result)

content='' additional_kwargs={} response_metadata={'model': 'qwen2.5:0.5b', 'created_at': '2025-05-22T15:57:43.766979053Z', 'done': True, 'done_reason': 'stop', 'total_duration': 405886742, 'load_duration': 15507290, 'prompt_eval_count': 185, 'prompt_eval_duration': 24054447, 'eval_count': 25, 'eval_duration': 361269271, 'model_name': 'qwen2.5:0.5b'} id='run--0ea15f71-c9cc-4499-984a-aef7840592e0-0' tool_calls=[{'name': 'big_crypto_coin', 'args': {'usd': 12}, 'id': '3906e7d6-1ab1-42b4-8992-f1c7d0c39b09', 'type': 'tool_call'}] usage_metadata={'input_tokens': 185, 'output_tokens': 25, 'total_tokens': 210}


As you can see, the LLM does not give us the answer. It provides a Tool call which we can use to get answer.

In [12]:
# And LLM suggest to make this Tool call
result.tool_calls

[{'name': 'big_crypto_coin',
  'args': {'usd': 12},
  'id': '3906e7d6-1ab1-42b4-8992-f1c7d0c39b09',
  'type': 'tool_call'}]

##### Lets try to make an automated way to make the tool calls. Since Tools calls is a list, we can loop over it and use the tool name and tool args to call tool

In [16]:
# First we need a mapping of Tool name with Tool's actual function so that we can call the function
# Followiung function can have more that 1 tool mapping, but we have only 1 right now. 
tool_map = {
    "big_crypto_coin": big_crypto_coin
}

for tool_info in result.tool_calls:
    tool = tool_map[tool_info['name']]
    tool_arg = tool_info['args']

    # Call the tool
    output = tool.invoke(tool_arg)
    print(output)

1482.72


### Langchain Builtin Tools

###### Langchain provides integration with wide variety of tools
[Check here - https://python.langchain.com/docs/integrations/tools/](https://python.langchain.com/docs/integrations/tools/)

For this learning, we will only use Wikipedia Tool

In [ ]:
!uv pip install wikipedia

In [8]:
# Lets see How wikipedia tool works
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper

# This is a wikipedia tool which can be given to an LLM
wiki = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# As we can see, it has these details
print(wiki.name)
print(wiki.description)
print(wiki.args)

wikipedia
A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
{'query': {'description': 'query to look up on wikipedia', 'title': 'Query', 'type': 'string'}}


In [9]:
# Lets try to run the tool
wiki.run("Avengers Infinity War")

"Page: Avengers: Infinity War\nSummary: Avengers: Infinity War is a 2018 American superhero film based on the Marvel Comics superhero team the Avengers. Produced by Marvel Studios and distributed by Walt Disney Studios Motion Pictures, it is the sequel to The Avengers (2012) and Avengers: Age of Ultron (2015), and the 19th film in the Marvel Cinematic Universe (MCU). Directed by Anthony and Joe Russo and written by Christopher Markus and Stephen McFeely, the film features an ensemble cast including Robert Downey Jr., Chris Hemsworth, Mark Ruffalo, Chris Evans, Scarlett Johansson, Benedict Cumberbatch, Don Cheadle, Tom Holland, Chadwick Boseman, Paul Bettany, Elizabeth Olsen, Anthony Mackie, Sebastian Stan, Danai Gurira, Letitia Wright, Dave Bautista, Zoe Saldaña, Josh Brolin, and Chris Pratt. In the film, the Avengers and the Guardians of the Galaxy attempt to stop Thanos from collecting the six powerful Infinity Stones as part of his quest to kill half of all life in the universe.\nTh

### Agents
We know that LLM cannot run a tool. It can only give details for invoking the tool.   
Agents bridge this gap. They get the info from LLM and invoke the Tool to generate the results.  
Lets create an agent which will call tools for us

In [5]:
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import (ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate)
from langchain.schema import SystemMessage
from langchain_ollama.chat_models import ChatOllama
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper

# LLM Instance
llm = ChatOllama(
    base_url = 'http://localhost:11434',
    model = 'qwen2.5:0.5b'
)

# Propmpt template
# We have 1 extra prompt at end which is agnet scratchpad for information coming from agent
prompt = ChatPromptTemplate.from_messages([
    SystemMessage("You are a question answering LLM with access to Wikipedia Tool. Before answering the question, use Wikipedia to get latest information and use this information to answer the question"),
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("Question: {question}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# Set of Tools 
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
tools = [wikipedia]

In [6]:
# Lest create an agnent
agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)

# This is just an agent, but we need an Agent executor to execute the agent correctly
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [15]:
# Lets use the agent to ask some questions
from langchain_core.output_parsers import StrOutputParser

result = agent_executor.invoke({'question': "What are Food specialities in Indore", 'chat_history': []})

print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Food specialities, Indore'}`


Page: Indore
Summary: Indore ( ; ISO: Indaura, Hindi: [ɪn̪d̪ɔːr]) is the largest and most populous city in the Indian state of Madhya Pradesh. It is the commercial hub of Madhya Pradesh. It is consistently ranked as the cleanest city in India. It serves as the headquarters of both the Indore District and the Indore Division. It is also considered the state education hub and houses campuses of both the Indian Institute of Technology and the Indian Institute of Management. Located on the southern edge of Malwa Plateau, at an average altitude of 553 metres (1,814 ft) above sea level, it has the highest elevation among major cities of Central India. The city is 220 km (137 mi) west of the state capital of Bhopal. Indore had a census-estimated 2011 population of 1,994,397 (municipal corporation) and 3,570,295 (urban agglomeration). The city is distributed over a land area of just 5